In [1]:
#import Pkg; 
#using Pkg;

#Pkg.add("ColorSchemes")
using Distributions, ColorSchemes, Statistics
  
# to create dataframes and load
#Pkg.add("DataFrames")
using DataFrames
  
# to use the above function kmeans()
#Pkg.add("Clustering")
using Clustering
  
# RDatasets to load the already made datasets
#Pkg.add("RDatasets")
using RDatasets

#Pkg.add("GR")
#Pkg.build("GR")
#import Pkg; Pkg.add("Plots")

using Plots, GeometryBasics, VoronoiCells
default(size=(800,600))

In [2]:
function rbf(x; weights=[1], centers=[0,0], sigma2=[1])
    
    @assert length(weights)==size(centers)[1]==length(sigma2)>0 #chequeo consistencia de n>0
    n=length(weights)

    @assert length(x)==length(centers[1]) #chequeo que los centros tengan la dim de x.
    d=length(x[1])
    
    kernels = [exp(-(x-centers[j])'*(x-centers[j])/sigma2[j]) for j=1:n]
    return sum(w.*kernels)
    
end

rbf (generic function with 1 method)

In [3]:
function integrate_voronoi(g, h, A_X, P)
    return A_X/P*sum(g.*h)
end

function g(x;w,mu,sigma2)
   return rbf(x, weights=w, centers=mu, sigma2=sigma2)
end

function h(x;w,mu,sigma2)
    return 1
end

function g_j(x;j,w,mu,sigma2)
   return rbf(x, weights=[w[j]], centers=[mu[j]], sigma2=[sigma2[j]])
end

function h0_j(x;j,w,mu,sigma2)
    return 1/w[j]
end

function h1_j(x;j,w,mu,sigma2)
    return (x-mu[j])'*(x-mu[j])/(sigma2[j]^4)
end

function h2_j(x;j,w,mu,sigma2)
    return  -2/sigma2[j]^2*(x-mu[j])
end

h2_j (generic function with 1 method)

In [4]:
function integrate(x, v_i, theta)
    
    w = theta[1]
    mu = theta[2]
    sigma2 = theta[3]

    # ========== Calculate differentials ========== 
    integrates = zeros(s)
    for i=1:s
        v = [[v_i[i][l,1], v_i[i][l,2]] for l=1:size(v_i[i])[1]];
        # For each station -> Integrates the RBF 
        g_out = g.(v, w=w, mu=mu, sigma2=sigma2)
        h_out = h.(v, w=w, mu=mu, sigma2=sigma2)
        integrates[i] = integrate_voronoi(g_out, h_out, A_X, P)
    end
    error_i = integrates - y
    
    # ========== Compute Loss differentials aproximationss ========== 
    n0_ij = zeros(s,n)
    n1_ij = zeros(s,n)
    n2x1_ij = zeros(s,n)
    n2x2_ij = zeros(s,n)

    for i=1:s
        v = [[v_i[i][l,1], v_i[i][l,2]] for l=1:size(v_i[i])[1]];
        for j=1:n
            # Evaluaciones
            g_j_out  = g_j.(v, j=j, w=w, mu=mu, sigma2=sigma2)
            h0_j_out = h0_j.(v, j=j, w=w, mu=mu, sigma2=sigma2)
            h1_j_out = h1_j.(v, j=j, w=w, mu=mu, sigma2=sigma2)
            h2_j_out = h2_j.(v, j=j, w=w, mu=mu, sigma2=sigma2)

            # Integrales
            n0_ij[i,j] = (integrate_voronoi(g_j_out, h0_j_out, A_X, P))
            n1_ij[i,j] = (integrate_voronoi(g_j_out, h1_j_out, A_X, P))
            n2x1_ij[i,j] = (integrate_voronoi(g_j_out, [h2_j_out[l][1] for l = 1:length(h2_j_out)], A_X, P))
            n2x2_ij[i,j] = (integrate_voronoi(g_j_out, [h2_j_out[l][2] for l = 1:length(h2_j_out)], A_X, P))
        end
    end
    
    # ========== Make the sums ========== 
    dL_w_j = zeros(n)
    dL_sigma2_j = zeros(n)
    dL_mux1_j = zeros(n)
    dL_mux2_j = zeros(n)

    for j = 1:n
        dL_w_j[j] = sum(error_i.*n0_ij[:,j])
        dL_sigma2_j[j] = sum(error_i.*n1_ij[:,j])
        dL_mux1_j[j] = sum(error_i.*n2x1_ij[:,j])
        dL_mux2_j[j] = sum(error_i.*n2x2_ij[:,j])
    end

    dL_mu_j = [[dL_mux1_j[i], dL_mux2_j[i]] for i = 1:3]
    
    dTheta = dL_w_j, dL_mu_j, dL_sigma2_j
    return dTheta
end

integrate (generic function with 1 method)

In [5]:
import Random
Random.seed!(100)

# METAPARAMETERS

s = 10 # Count of sites 
d = 2 # Space Dimension
n = 3; # Count of kernels
P = 3000; # Count of auxiliar points
norm = 2; # Norm to use
nu_min = 0.05; # Min gradient step
nu_max = 0.99; # Max gradient setp
nu_decay = 0.99; 
integration_max_steps = 25000;
integration_max_error = (0.001, [0.005, 0.005], 0.001); # (w, [mux1, mux2], sigma2)

# R^d limits

Xmin = [-1 -1];
Xmax = [1 1];
Deltas = Xmax - Xmin

# Sites

x = sortslices(rand(Uniform(0,1),s,d), dims = 1) # Sites coordinates in R^d
x1 = x[:,1] = x[:,1].*Deltas[1].+Xmin[1] #TODO: Generalize to d
x2 = x[:,2] = x[:,2].*Deltas[2].+Xmin[2] #TODO: Generalize to d
y = rand(s); # Sites demands

# Auxiliar points to integrate

X = sortslices(rand(Uniform(0,1),P,d), dims = 1); # Auxiliar points to integrate
X1 = X[:,1] = X[:,1].*Deltas[1].+Xmin[1]
X2 = X[:,2] = X[:,2].*Deltas[2].+Xmin[2]

A_X = Deltas[1]*Deltas[2]; # Limits area

# Voronoi assigments

W = [(sum((x[i,:]-X[k,:]).^norm))^(1/norm) for i=1:s,k=1:P];
attachs = zeros(P)
for k=1:P    
    _,idx = findmin(W[:,k])
    attachs[k] = idx
end
attachs = Int64.(attachs);
v_i = [ X[findall(x->x==i,attachs),:] for i in 1:s ]; 

In [6]:
# THETA INITIALIZATION 

w = [1/(2*pi) for j=1:n]; #Pesos iniciales (TODO: paso C inicialización)

R = kmeans(x', n, weights = y, maxiter = 200);#, display=:iter);
mu = [[R.centers'[i,1],R.centers'[i,2]] for i=1:n];
mu1 = R.centers'[:,1];
mu2 = R.centers'[:,2];

sigma2 = [var(x[findall(x->x==i,assignments(R)),:]) for i in 1:n];

theta0 = w, mu, sigma2
theta = theta0

([0.15915494309189535, 0.15915494309189535, 0.15915494309189535], [[-0.03688696876779144, 0.7453183720655502], [-0.7258338519917173, -0.7194806446467097], [-0.06096357482135974, -0.16977691886924512]], [0.27902401737024773, 0.04403032144685578, 0.18602921686475637])

In [7]:
# INTEGRATION AND GRADIENT STEP

nu = nu_max

for k = 1:integration_max_steps
    nu = max(nu*nu_decay, nu_min)
    dTheta = integrate(x, v_i, theta)
    oldTheta = theta;
    theta = theta .- nu.*dTheta;
    
    # Comprobar umbrales
    condition = (maximum.(theta .- oldTheta) .< integration_max_error) # | (maximum.(oldTheta .- theta) .< integration_max_error)
    if condition == (true, true, true)
        println("cortar por umbral en ", k, " iteraciones")
        k = integration_max_steps
        break
    end
end

cortar por umbral en 13598 iteraciones


In [36]:
(theta .- theta0)

([6.251393518825113, 25.625617927939196, 18.844951588378777], [[1.98676476436113, 4.230639435909844], [-0.16681623294972758, 0.49790082208505826], [4.054994958100723, -2.0909032970700747]], [4.457726099616781, 380.3792787711335, 12.62920190338326])

In [31]:
theta0

([0.15915494309189535, 0.15915494309189535, 0.15915494309189535], [[-0.03688696876779144, 0.7453183720655502], [-0.7258338519917173, -0.7194806446467097], [-0.06096357482135974, -0.16977691886924512]], [0.27902401737024773, 0.04403032144685578, 0.18602921686475637])

In [9]:
theta

([6.410548461917008, 25.78477287103109, 19.004106531470672], [[1.9498777955933384, 4.975957807975394], [-0.8926500849414449, -0.22157982256165146], [3.9940313832793635, -2.26068021593932]], [4.736750116987029, 380.4233090925804, 12.815231120248017])